In [2]:
import os
import urllib
from urllib2 import urlopen, URLError, HTTPError
import zipfile
from bs4 import BeautifulSoup
import re

import pycurl
import re
import urllib2

import fiona
import geopandas as gpd
import glob
from rasterstats import zonal_stats
import pandas as pd
import pickle

from library.geoprocess import get_tif_ids, zonal_to_shp
from library.utilities import mkdir, dlfile, unzip_file

In [3]:
# make data, shp, and countries directories if they don't already exist
mkdir('data/shp/countries')

In [40]:
# get country shapefiles zipfile
url = 'http://biogeo.ucdavis.edu/data/world/countries_shp.zip'
dlfile(url, 'data/shp/countries', 'countries_shp.zip')

downloading http://biogeo.ucdavis.edu/data/world/countries_shp.zip
data/shp/countries/countries_shp.zip
countries_shp.zip


In [5]:
# unzip shapefiles
unzip_file('data/shp/countries/countries_shp.zip', 'data/shp/countries')

In [6]:
# read in countries shapefile
shp = 'data/shp/countries/countries.shp'
gdf = gpd.GeoDataFrame.from_file(shp); gdf.head(3)

,CONTINENT,COUNTRY,ENGLISH,EU,FAO,FIPS,FRENCH,ISO2,ISO3,LOCAL,NAME,OBJECTID,SOVEREIGN,SPANISH,SQKM,UNREG1,UNREG2,WAS_ISO,geometry
0,Europe,Åland,Åland,0,None,AX,None,AX,ALA,Åland,Åland,1,Finland,None,1243.719143,Northern Europe,Europe,None,(POLYGON ((20.99566650547376 60.64227676165353...
1,Asia,Afghanistan,Afghanistan,0,Afghanistan,AF,Afghanistan,AF,AFG,Afghanestan,Afghanistan,2,Afghanistan,Afganistán,641383.422787,Southern Asia,Asia,None,"POLYGON ((73.2733612030425 36.88855743734217, ..."
2,Europe,Albania,Albania,0,Albania,AL,Albanie,AL,ALB,Shqiperia,Albania,3,Albania,Albania,28486.109599,Southern Europe,Europe,None,(POLYGON ((20.98056793146918 40.85522079417342...


In [44]:
# download lights data
# see http://ngdc.noaa.gov/eog/dmsp/downloadV4composites.html for webpage that lists links

# make img directory
mkdir('data/img')

# build links list
webpage_url = 'http://ngdc.noaa.gov'
base_url = 'http://ngdc.noaa.gov/eog/dmsp/downloadV4composites.html'
soup = BeautifulSoup(urlopen(base_url), 'lxml')

links_list = []
for i in soup.find_all('a'):
    link = i.get('href')
    links_list.append(link)

links_list = filter(None, links_list)
links_list = [x for x in links_list if '.tar' in x]
links_list = [x for x in links_list if '_x_pct' not in x]

links_list_cleaned = []
for i in links_list:
    replacement = i.replace('\n', '')
    links_list_cleaned.append(replacement)

links_list_complete = []
for i in links_list_cleaned:
    link = webpage_url + i
    links_list_complete.append(link)

# make list of filenames
filenames = []
for link in links_list_cleaned:
    a_file = link.split('/eog/data/web_data/v4composites/', 1)[1]
    filenames.append(a_file)

# zip links and filenames lists together
links_filenames = zip(links_list_complete, filenames)

# get raster tarballs; this will take awhile
for tu in links_filenames:
    dlfile(tu[0], 'data/img', tu[1])

downloading http://ngdc.noaa.gov/eog/data/web_data/v4composites/F101992.v4.tar
data/img/F101992.v4.tar
F101992.v4.tar
downloading http://ngdc.noaa.gov/eog/data/web_data/v4composites/F101993.v4.tar
data/img/F101993.v4.tar
F101993.v4.tar
downloading http://ngdc.noaa.gov/eog/data/web_data/v4composites/F101994.v4.tar
data/img/F101994.v4.tar
F101994.v4.tar
downloading http://ngdc.noaa.gov/eog/data/web_data/v4composites/F121994.v4.tar
data/img/F121994.v4.tar
F121994.v4.tar
downloading http://ngdc.noaa.gov/eog/data/web_data/v4composites/F121995.v4.tar
data/img/F121995.v4.tar
F121995.v4.tar
downloading http://ngdc.noaa.gov/eog/data/web_data/v4composites/F121996.v4.tar
data/img/F121996.v4.tar
F121996.v4.tar
downloading http://ngdc.noaa.gov/eog/data/web_data/v4composites/F121997.v4.tar
data/img/F121997.v4.tar
F121997.v4.tar
downloading http://ngdc.noaa.gov/eog/data/web_data/v4composites/F141997.v4.tar
data/img/F141997.v4.tar
F141997.v4.tar
downloading http://ngdc.noaa.gov/eog/data/web_data/v4com

KeyboardInterrupt: 